### pendulum : datetime 조작 library
- pip install pendulum

In [1]:
import pendulum
dt = pendulum.datetime(2023,11,24, tz="Asia/Seoul")
print(type(dt),dt)

<class 'pendulum.datetime.DateTime'> 2023-11-24T00:00:00+09:00


In [7]:
import oracledb
import os
import pandas as pd

# dsn = cx_Oracle.makedsn(host=hostName, port=partNumber, sid=None, service_name=serviceName)

un = os.environ.get('PYTHON_USERNAME')
pw = os.environ.get('PYTHON_PASSWORD')
cs = os.environ.get('PYTHON_CONNECTSTRING')

# oracledb.init_oracle_client(lib_dir="/Desktop/oracle/instantclient_19/basic")
oracledb.init_oracle_client()
con = oracledb.connect(user="hmuser", password='hmpass', host="localhost", port=1521, service_name="XE")
cursor = con.cursor()

#쿼리문
sql = """select * from blog"""

cursor.execute(sql)

columns = [col[0] for col in cursor.description]

x = cursor.fetchall()
df = pd.DataFrame(x, columns=columns)

cursor.close()
con.close()

In [8]:
df.head()

,BID,PENNAME,TITLE,CONTENT,MODTIME,VIEWCOUNT,ISDELETED
0,1,고수,오라클,오라클은 참 좋은 DMBS이네요.,2023-08-23 14:19:23.354,3,0
1,2,고수,MYSQL,MYSQL도 참 좋은 DMBS이네요.,2023-08-23 14:19:42.179,0,1
2,3,프로그래머,자바,자바는 객체지향 프로그램...\r\n컬렉션도 중요해요...,2023-08-25 10:15:40.220,9,0
3,4,프로그래머,파이썬,파이썬은 배우기 쉬운 프로그래밍 언어입니다.,2023-08-25 10:14:37.205,2,0
4,6,스프링,스프링 부트,김수정\r\n양수정,2023-08-24 16:23:55.858,46,1


In [41]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [42]:
# 필요한 라이브러리를 불러옵니다.
def pyarrowDf_to_db_executemany(df, chunksize):
    import os  # 운영 체제와 상호 작용하기 위한 라이브러리
    import sys  # Python 시스템 관련 기능을 사용하기 위한 라이브러리
    import pendulum  # 날짜와 시간을 다루기 쉽게 해주는 라이브러리
    import oracledb  # Oracle 데이터베이스와 상호 작용하기 위한 라이브러리

    # 함수 이름과 실행 시간을 출력합니다.
    print("[function name]: [%s]: [%s]" % (sys._getframe().f_code.co_name, pendulum.now('Asia/Seoul').strftime('%Y-%m-%d %H:%M:%S')))

    # 함수 실행 시작 시간을 기록합니다.
    start_dt = pendulum.now('Asia/Seoul')

    # Oracle 데이터베이스 연결 정보를 환경 변수에서 가져옵니다.
    un = os.environ.get('PYTHON_USERNAME')
    pw = os.environ.get('PYTHON_PASSWORD')
    cs = os.environ.get('PYTHON_CONNECTSTRING')

    # Oracle 클라이언트를 초기화하고 데이터베이스에 연결합니다.
    oracledb.init_oracle_client()
    con = oracledb.connect(user="hmuser", password='hmpass', host="localhost", port=1521, service_name="XE")
    cur = con.cursor()

    # 데이터베이스 테이블을 초기화하는 주석 처리된 코드입니다.
    # cur.execute("TRUNCATE TABLE PINO93.NHIS_OPEN_T60_2018")
    # con.commit()

    # 데이터베이스에 삽입할 SQL 문을 작성합니다.
    insert_statement = """INSERT INTO RENT (week, sta_id, addr, no_rent) VALUES (:1, :2, :3, :4)"""

    # SQL 문을 미리 컴파일하여 성능을 최적화합니다.
    try:
        cur.prepare(insert_statement)
    except:
        print('insert_statement Error')
        exit(1)

    # 주어진 데이터프레임을 작은 청크로 나누어 반복적으로 처리합니다.
    for subDf in chunker(df, chunksize):
        # 데이터프레임을 리스트로 변환합니다.
        csv_df_dataset_lst = subDf.values.tolist()
        # executemany를 사용하여 여러 데이터를 한 번에 삽입합니다.
        cur.executemany(insert_statement, csv_df_dataset_lst)
        # 변경사항을 데이터베이스에 반영합니다.
        con.commit()

    # 커서와 데이터베이스 연결을 닫습니다.
    cur.close()
    con.close()

    # 함수 실행 종료 시간을 기록하고 소요된 시간을 출력합니다.
    end_dt = pendulum.now('Asia/Seoul')
    delta = end_dt - start_dt
    print("[function]:[%s], [chunksize]:[%s], [elapsed time]:[%s]" % (sys._getframe().f_code.co_name, format(chunksize, ','), delta.in_words()))


In [11]:
import pandas as pd
df = pd.read_csv('../Pro/data/2023/2023(1~6).csv')
df.head()

,주차,대여소번호,대여소,대여건수
0,1,102,102. 망원역 1번출구 앞,171.0
1,1,103,103. 망원역 2번출구 앞,157.0
2,1,104,104. 합정역 1번출구 앞,141.0
3,1,105,105. 합정역 5번출구 앞,71.0
4,1,106,106. 합정역 7번출구 앞,145.0


In [12]:
df.shape

(83862, 4)

In [13]:
df.dtypes

주차        object
대여소번호     object
대여소       object
대여건수     float64
dtype: object

In [20]:
df.isna().sum()

주차           0
대여소번호    67322
대여소      67770
대여건수     67770
dtype: int64

In [26]:
df.tail(5)

,주차,대여소번호,대여소,대여건수
83857,26\t5866\t5866.포레나 당산\t165,NaN,NaN,NaN
83858,26\t5867\t5867. H타워 앞\t156,NaN,NaN,NaN
83859,26\t6053\t6053. 중부세무서 앞\t31,NaN,NaN,NaN
83860,26\t6054\t6054.청구e편한세상아파트 113동 앞\t123,NaN,NaN,NaN
83861,26\t9980\t9980. 에이텍\t1,NaN,NaN,NaN


In [28]:
df.dropna(how='any', inplace=True)
df.shape

(16092, 4)

In [29]:
df.tail(5)

,주차,대여소번호,대여소,대여건수
16087,52,5859,5859. 여의도 삼부아파트,10.0
16088,52,5860,5860. 어울림병원 앞,7.0
16089,52,5861,5861. 보라주유소 앞,4.0
16090,52,5862,5862. 당산역11번출구,9.0
16091,52,6053,6053. 중부세무서 앞,2.0


In [30]:
df['대여건수'] = df['대여건수'].astype('int')
df

,주차,대여소번호,대여소,대여건수
0,1,102,102. 망원역 1번출구 앞,171
1,1,103,103. 망원역 2번출구 앞,157
2,1,104,104. 합정역 1번출구 앞,141
3,1,105,105. 합정역 5번출구 앞,71
4,1,106,106. 합정역 7번출구 앞,145
...,...,...,...,...
16087,52,5859,5859. 여의도 삼부아파트,10
16088,52,5860,5860. 어울림병원 앞,7
16089,52,5861,5861. 보라주유소 앞,4
16090,52,5862,5862. 당산역11번출구,9


In [31]:
df.head()

,주차,대여소번호,대여소,대여건수
0,1,102,102. 망원역 1번출구 앞,171
1,1,103,103. 망원역 2번출구 앞,157
2,1,104,104. 합정역 1번출구 앞,141
3,1,105,105. 합정역 5번출구 앞,71
4,1,106,106. 합정역 7번출구 앞,145


- 대여소 마스터 DB와 CSV 이름 매칭
- 대여소번호	보관소(대여소)명	자치구	상세주소	위도	경도	설치시기	도로명주소	법정동	행정동
- 'sta_id', 'sname', 'gu', 'addr1', 'lat', 'lng', 'install_date', 'addr2', 'dong1', 'dong2'


#### 컬럼 이름 영어로 변경 - 정류소 master

In [32]:
df.columns = ['week', 'sta_id', 'addr', 'no_rent']
df.head()

,week,sta_id,addr,no_rent
0,1,102,102. 망원역 1번출구 앞,171
1,1,103,103. 망원역 2번출구 앞,157
2,1,104,104. 합정역 1번출구 앞,141
3,1,105,105. 합정역 5번출구 앞,71
4,1,106,106. 합정역 7번출구 앞,145


In [38]:
df.dtypes

week        int32
sta_id      int32
addr       object
no_rent     int32
dtype: object

In [37]:
df = df.astype({'week':'int', 'sta_id':'int'})

In [34]:
df.isna().sum()

week       0
sta_id     0
addr       0
no_rent    0
dtype: int64

In [35]:
df.shape

(16092, 4)

In [36]:
df.nunique()

week          6
sta_id     2708
addr       2708
no_rent     300
dtype: int64

In [15]:
df.shape

(2137, 10)

In [43]:
pyarrowDf_to_db_executemany(df, 5000)

 [function name] : [pyarrowDf_to_db_executemany] : [2023-12-06 17:55:10] 
[function]:[pyarrowDf_to_db_executemany], [chunksize]:[5,000], [elapsed time]:[0.09 second]
